In [1]:
pip install datasets

In [2]:
pip install --upgrade openai

In [3]:
# Imports
import json
import re
import os
import numpy as np
import pandas as pd
from openai import OpenAI
from datasets import load_dataset

from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPEN_API_KEY_IR'))

# Dataset

In [4]:
#load the test data for the cross-model evaluation of the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "cross_model_evaluation")

print(data.keys())

data_de_en = data['cross_model_evaluation_de_en'].to_pandas()
data_en_de = data['cross_model_evaluation_en_de'].to_pandas()
data_fr_en = data['cross_model_evaluation_fr_en'].to_pandas()
data_en_fr = data['cross_model_evaluation_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


dict_keys(['cross_model_evaluation_de_en', 'cross_model_evaluation_en_de', 'cross_model_evaluation_fr_en', 'cross_model_evaluation_en_fr'])


,id,source,hyp1,hyp2
0,0,"""Perry erklärte, er werde „nach Texas zurückke...",Perry declared that he would'return to Texas t...,Perry declared that he would return to Texas t...
1,1,"Bachmann, die im August die Ames-Probeumfrage ...","Bachmann, who won the Ames straw poll in Augus...","Bachmann, who won the Ames straw poll in Augus..."
2,2,Die Kruste ist auf der nahen Seite etwa 70 km ...,The crust is on the nearby side about 70 km th...,The crust is approximately 70 km thick on the ...
3,3,Der Anwalt forderte die Wirtschaftsprüferin au...,The lawyer demanded that the auditor provide d...,The lawyer demanded the auditor to present the...
4,4,"Diese Art ist in der Schweiz, in Frankreich un...",This type of art is represented in Switzerland...,This type of art is represented in Switzerland...


,id,source,hyp1,hyp2
0,0,The pirates were attacking the ship in order t...,"Die Piraten griffen das Schiff an, um die Prin...","Die Piraten griffen das Schiff an, um die Prin..."
1,1,"Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, ein Mathematiker und Informatik..."
2,2,In the summer of 1924 he went to Smackover in ...,Im Sommer 1924 machte er Urlaub in Smackover i...,Im Sommer 1924 ging er nach Smackover in Süd-A...
3,3,I am informed that yesterday evening a frigate...,"1. Ich bin informiert, dass gestern Abend ein ...","Ich erfuhr, dass gestern Abend ein Fregattenbo..."
4,4,"Heels should be low and wide. Sand, gravel or ...","Absätze sollten niedrig und breit sein. Sand, ...",Absätze sollten sehr niedrig und sehr breit se...


## Generate Data

In [5]:
def detect_hallu(source, hyp1, hyp2):

  answer_format = {"label": ""}

  system_msg = "Imagine you are a classifier"

  # prompt for translation detection task
  user_msg = f'''Given a "src" and two hypotheses "hyp1" and "hyp2" your task is to detect which of the two hypotheses is hallucinated ("label").
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''

  response = client.chat.completions.create(model="gpt-3.5-turbo",
                                        messages=[{"role": "system", "content": system_msg},
                                         {"role": "user", "content": user_msg}])

  detected_hallu = response.choices[0].message.content

  return detected_hallu

## Save Data as csv file

In [6]:
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [7]:
#save results
def save_results(df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallu(source, hyp1, hyp2)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows

# Translation German - English

In [8]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_de_en.csv"

skipped_rows_de_en = save_results(data_de_en, csv_filename)

In [9]:
skipped_rows_de_en

[]

# Translation English - German

In [10]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_en_de.csv"

skipped_rows_en_de = save_results(data_en_de, csv_filename)

# Translation French - English

In [11]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_fr_en.csv"

skipped_rows_fr_en = save_results(data_fr_en, csv_filename)

In [ ]:
answer_format = {"type": "", "label": ""}

system_msg = "Imagine you are a classifier"

 # In dem Beispiel ist ein j zu viel aber er erkennt das irgendwie nicht???
user_msg = f'''Given a "src" and two hypotheses "hyp1" and "hyp2" your task is to detect which of the two hypotheses is hallucinated ("label") and what type of hallucination ("type") it is.
      Provide the result in the following format: {answer_format}.

       Src: Marshall Scott III est surintendant depuis le 1er juin 2017, la directrice des études est Lisa Moya, le directeur de la technologie Josh Solis.
       hyp1 : Dr. Marshallj Scott III has been superintendent since June 1, 2017. Chief Academic Officer is Lisa Moya. Chief Technology Officer is Josh Solis.
       hyp2 : Dr. Marshall Scott III has been superintendent since June 1, 2017. Chief Academic Officer is Lisa Moya. Chief Technology Officer is Josh Solis.

       Result:
       '''

response = client.chat.completions.create(model="gpt-3.5-turbo",
                                        messages=[{"role": "system", "content": system_msg},
                                         {"role": "user", "content": user_msg}])

#print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
print(response.choices[0].message.content)

{'type': '', 'label': ''}


# Translation English - French

In [12]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/gpt3.5/results_gpt_en_fr.csv"

skipped_rows_en_fr =  save_results(data_en_fr, csv_filename)